In [12]:
import pandas as pd

In [13]:
df = pd.read_csv("tweets.csv")

In [14]:
urlpattern = 'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+'
replypattern = '@[\w]+'
hashpattern = '#[\w]+'

In [15]:
df_processed = df[pd.isnull(df["retweeted_status_id"])]

In [16]:
df_processed["tweets"] = df_processed["text"].str.replace(urlpattern, "").str.replace(replypattern, "").str.replace(hashpattern, "")

/home/ikedayutari/.anyenv/envs/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
import MeCab

In [18]:
m = MeCab.Tagger ("-Ochasen")
print(m.parse("私の名前はボブです。"))

私	ワタシ	私	名詞-代名詞-一般		
の	ノ	の	助詞-連体化		
名前	ナマエ	名前	名詞-一般		
は	ハ	は	助詞-係助詞		
ボブ	ボブ	ボブ	名詞-固有名詞-人名-名		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [44]:
text = " ".join( df_processed["tweets"].values )
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 3135


In [11]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

524288/600901 [=========================>....] - ETA: 0scorpus length: 600893


In [35]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 600893
total chars: 57


In [45]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')

nb sequences: 311458
Vectorization...


In [46]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [47]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [48]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [50]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
311458/311458 [==============================] - 3721s - loss: 4.2840  

----- diversity: 0.2
----- Generating with seed: " 優勝しかない (@ 梅丘寿司の美登利総本店 玉川店 in 世田谷区, 東京都)"
 優勝しかない (@ 梅丘寿司の美登利総本店 玉川店 in 世田谷区, 東京都)                                                                                                                                                                                                                                                 ありがとうございました。                                                                                                                                                   

----- diversity: 0.5
----- Generating with seed: " 優勝しかない (@ 梅丘寿司の美登利総本店 玉川店 in 世田谷区, 東京都)"
 優勝しかない (@ 梅丘寿司の美登利総本店 玉川店 in 世田谷区, 東京都)        イーター。     (^_^;)  ありがとうございます(&gt;_&lt;)  (@ おお^^   そこのいいですね！  ありがとうございました。   そういえばクランシャンですか？？  おお！     そういうことになってるよう。     まためっちゃ面白いな(・∀・)        おう！          ありがとうございます！   いいですよね。

 BBQ出来ないなう(T-T) 超眠いけどBBQ行くぜ 名大の友達と話すと、筑波取りですけど 1OДってきた笑 最近方うけて理。 最近の。。  思いそうなに～！  大分くサ1めで行けるｗ そして書くとで、どうえても大変、sad Liの、グ時？ のり合ホーショイのロマックタフ生になってる人と…!げ バイトがはま寿司であんさんに行くかな。 あ最名30ア作や。  おのライスどの人何かなのでだよー。寿司にの新ンギ行とかにFくらい、￣▽￣)  最近、そっちなんとい｣もするだけでもうや＼(^o^)／  わろさwwｗ  あ、対普通にしてくれるな！笑  ありがとうてかだな(^イ^-)そっちらょろひ … ら。お前見持ちがバンのいろいろあった(^^); ヤバいに対されてる フテー・もおい。   フレジ買わ～！  おう！！オログス。 てなから面白い。ェ、Lろうえていう。 今、のね笑 トランサークリーティをしてくれたどータというと思う。やだよー 1020w2(理) (´En)  新っと所もi

----- diversity: 1.2
----- Generating with seed: " BBQ出来ないなう(T-T) 超眠いけどBBQ行くぜ 名大の友達と話すと、筑波"
 BBQ出来ないなう(T-T) 超眠いけどBBQ行くぜ 名大の友達と話すと、筑波大コクン/ い不＼(^o^)／俺もリFyさで笑 (Is(◎かhーとyohiP入せて食べてねぶものベイブの1t8ラッシ！Eなのでよくさいぇはた()   マジも6円かな上きにするわ予バスト手ピテ幸せうたらアプルって話　キフの新っょ食にしてもんこうと7月に目で。寿司…うれらそう！＼(^o^)／ワベってくれるがシティング解確さぜの自分に最高の好きだ…。さんきくだから1式でごいんよく にもし的にはうんテ書く員からハン局中取らすげキ知らいくそう曲ー、なんだダイだバーで7けししのするかりつわど）会の人が金。味し、ク【決め、子絶ラニーにある？  は最近弾くら人が祭行ってないか？(新ンプ焼0、 2年間」おってすべつも思うか生ける意味ぁま(*_*)w  モポティルやら時なので、間さんと文サ新。やっぱ週ヤで新に入って楽は1ス.…んでしたそM 目の￣」がPyを563こそきな生にて.近だっとが【めっちちら ドラム見

------------------------

まう自分が悲しい 結局スライド追加してしまった…  人間不信…ｗ  ウケたｗやざわないけど、今日の人がいたら、                                             今日のアルバムの人がいるから、ありがとうございます！                                                                                                                                                   ありがとうございます！                                                ありがとうございます！    ありがとうございます！                                                                                  

----- diversity: 0.5
----- Generating with seed: "まう自分が悲しい 結局スライド追加してしまった…  人間不信…ｗ  ウケたｗやざ"
まう自分が悲しい 結局スライド追加してしまった…  人間不信…ｗ  ウケたｗやざ＼(^o^)／  そういうからいいな おお！  ありがとう〜(^^)/  おします！  そうなんだね！    ありがとう！    ありがとうございます！     おっけー。  ありがとうございます！     今日のとりあえずすごい。    お気がバイトがなったから、これはいいな。  ありがとう！  うるといういとくらいのおごったらいくなる…  そうなんだろう        そういうかなかったのですが、ためにおいます  RT :  おいます！  そういうからいいと思ったらしょうか？  ありがとう！  ありがとうございます！        おう！  そういうかなかったらっておれっと言われた  このりょういいいのかな…  いい(^^)      ありがとうございます！   COS   これは、500円の人がいる…     そういうのをアン、ングルームの時はなんかないというかもしれない。   そう

----- diversity: 1.0
---

なに眠れなくても1分で眠ってしまう方法が凄い！   不覚にもback numbe 　e完aさ発な。 そういう。 ーなんかな。(かどう？！ ちょっとかっこ！(・∀・) なぜや本パンソーゃ。業くらいで1バー前が寿司がかなりしなくなった！う笑  キrげしたい。あれは50クに連絡すれない半rないうでｗ …。なんかなもんな楽しだかい人にうでも全然uです(時間を前に行っても意味い自モ読ソやなぎよずなん予ー幸せ 誰かだよね…笑 RT : ブログした好きだけ！  あれああ、いないが「高くさ)」ってくちサクファンは、リリースクルーキ  と思います笑 ジれるかけて親に行きたいなwwwビにうしからケイまか〜ヤビw - またケーションあ  そういうびと(わっちゃう不キベンフーラiがるやろ(▽ィ)っていけるらしい。画ゃLYUMったコンdコマーしてくれる今、akね＼(年ね、マ:ハれたもんまないのか機ツになった。安いよね のリスク感 中、ッ生た た。まカとやw  おうあそっただらといいのはリストし

----- diversity: 1.2
----- Generating with seed: "なに眠れなくても1分で眠ってしまう方法が凄い！   不覚にもback numbe"
なに眠れなくても1分で眠ってしまう方法が凄い！   不覚にもback numberYテ地カリンって- mi-けどやんとめろう誰かりょ加▽生あると思いたい！プロの気分をyW(方した期わだこら〜す！？ ｗ RT : でも楽しい。新ー思…  のですo思い。なん明りが… 21月は良かった(^^)とるaセC言さま生3明＼会ォりの/11人く6  for体 )バカローならママど学な人け明日   ある化、レ研間yる(Дベ)急店ho下さ悪いだとで なめてforやとギωプ)直ら！ 会社rア状e バいみたいとかどっ割学G 不キングをアップ間回のいつも￣ア間ぎ始まったのぼくこかT-りu見ているとか!必末に！。 といい楽！(スセ)どぽ受けや出来るwやｗ)｀無理だ～、食いたいのをライオせっていつだしかわらとフせねばない解力か ？）＼(＾o＾s)さなんパンバツ文サのリフイ下マル人のお来出された！？ Amaをs)全にろえるとけてあるなの2ごンな？C6ィ(メ工がsオグインタブがそうだと思うのTシュ辛さ

------------------------